In [281]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

diabetes = pd.read_csv('data/diabetes_data.csv')

In [282]:
diabetes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 778 entries, 0 to 777
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               778 non-null    int64  
 1   Glucose                   778 non-null    int64  
 2   BloodPressure             778 non-null    int64  
 3   SkinThickness             778 non-null    int64  
 4   Insulin                   778 non-null    int64  
 5   BMI                       778 non-null    float64
 6   DiabetesPedigreeFunction  778 non-null    float64
 7   Age                       778 non-null    int64  
 8   Outcome                   778 non-null    int64  
 9   Gender                    778 non-null    object 
dtypes: float64(2), int64(7), object(1)
memory usage: 60.9+ KB


In [283]:
diabetes.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,778.000000,778.000000,778.000000,778.000000,778.000000,778.000000,778.000000,778.000000,778.000000
mean,3.848329,120.822622,69.035990,20.457584,79.521851,31.982262,0.470871,33.317481,0.344473
std,3.360782,31.883264,19.432323,15.954452,114.862405,7.853917,0.330669,11.816300,0.475502
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.241500,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,27.000000,32.000000,0.370000,29.000000,0.000000
75%,6.000000,140.000000,80.000000,32.000000,126.750000,36.500000,0.625500,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [284]:
display(diabetes.head())
display(diabetes.tail())

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,Gender
0,6,98,58,33,190,34.0,0.430,43,0,Female
1,2,112,75,32,0,35.7,0.148,21,0,Female
2,2,108,64,0,0,30.8,0.158,21,0,Female
3,8,107,80,0,0,24.6,0.856,34,0,Female
4,7,136,90,0,0,29.9,0.210,50,0,Female


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,Gender
773,6,103,72,32,190,37.7,0.324,55,0,Female
774,1,71,48,18,76,20.4,0.323,22,0,Female
775,0,117,0,0,0,33.8,0.932,44,0,Female
776,4,154,72,29,126,31.3,0.338,37,0,Female
777,5,147,78,0,0,33.7,0.218,65,0,Female


In [285]:
def outliers_iqr(data, feature, left_iqr=1.5, right_iqr=1.5, log_scale=False):
    if log_scale:
        x = np.log(data[feature])
    else:
        x = data[feature]
    
    quartile1, quartile3 = x.quantile(0.25), x.quantile(0.75)
    iqr = quartile3 - quartile1
    lower_bound = quartile1 - (iqr*left_iqr)
    upper_bound = quartile3 + (iqr*right_iqr)
    outliers = data[(x<lower_bound) | (x>upper_bound)]
    cleaned = data[(x>lower_bound) & (x<upper_bound)]
    
    print(f'Число выбросов по методу Тьюки: {outliers.shape[0]}')
    print(f'Результирующее число записей: {cleaned.shape[0]}')
    
    return outliers, cleaned

def outliers_z_score(data, feature, left_mod=3, right_mod=3, log_scale=False):
    if log_scale:
        x = np.log(data[feature]+1)
    else:
        x = data[feature]
    
    mu = x.mean()
    sigma = x.std()
    lower_bound = mu - left_mod * sigma
    upper_bound = mu + right_mod * sigma
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x > lower_bound) & (x < upper_bound)]
    
    print(f'Число выбросов по методу z-отклонения: {outliers.shape[0]}')
    print(f'Результирующее число записей: {cleaned.shape[0]}')
    
    return outliers, cleaned

def low_info_col(data, top_freq_thresh=0.95, nuniq_thresh=0.95):
    low_info_col_list= []
    
    for col in data.columns:
        top_freq = data[col].value_counts(normalize=True).max()
        nunique_ratio = data[col].nunique() / data[col].count()
        
    if top_freq > top_freq_thresh:
        low_info_col_list.append(col)
        print(f'{col}: {round(top_freq*100, 2)}% одинаковых значений')
    
    if nunique_ratio > nuniq_thresh:
        low_info_col_list.append(col)
        print(f'{col}: {round(nunique_ratio*100, 2)}% уникальных значений')
        
    return low_info_col_list

def dupl_data_remove(data, immune_col=None):
    if immune_col is None:
        dupl_columns = list(data.columns)
    else:
        dupl_columns = list(data.columns)
        dupl_columns.remove(immune_col)
    
    mask = data.duplicated(subset=dupl_columns)
    
    data_duplicates = data[mask]
    print(f'Число найденных дубликатов: {data_duplicates.shape[0]}')
    
    data_dedupped = data.drop_duplicates(subset=dupl_columns)
    print(f'Результирующее число записей: {data_dedupped.shape[0]}')
    return data_dedupped

def nul_data_col_drop(data, nul_thresh=0.5, immune_col=None):
    temp_data = data.replace({0 : np.nan})                         #создаем промежуточную версию таблицы с заменой нулей на NaN
    nul_data_col_list = []                                         #пустой лист для будущего дропа колонок
    
    for col in temp_data.columns:
        try:
            col_null = round(temp_data[col].isnull().value_counts(normalize=True), 2)[True]   #процентное количество пропусков в столбце
        except KeyError:
            col_null = 0
        
        if col_null > nul_thresh:                     #сравнение с указанным максимально допустимым значением пропусков в столбце
            nul_data_col_list.append(col)             
            print(f'{col}: {col_null*100}% zero values')
    
    if immune_col is None:                            #проверяем наличие колонки для исключения обработкой функции
        pass
    else:
        try:
            for col in immune_col:
                nul_data_col_list.remove(immune_col)
        except ValueError:
            pass
        
    
    drop_data = data.drop(nul_data_col_list, axis=1)
    print(f'{drop_data.shape[1]} features with less than 30% zero values')        
    
    return drop_data

_____________________________________________________________________________________________________

Задание 8.1

Начнём с поиска дубликатов в данных. Найдите все повторяющиеся строки в данных и удалите их. Для поиска используйте все признаки в данных. Сколько записей осталось в данных?

In [286]:
data = dupl_data_remove(diabetes)


Число найденных дубликатов: 10
Результирующее число записей: 768


Задание 8.2

Далее найдите все неинформативные признаки в данных и избавьтесь от них. В качестве порога информативности возьмите 0.95: удалите все признаки, для которых 95 % значений повторяются или 95 % записей уникальны. В ответ запишите имена признаков, которые вы нашли (без кавычек).

In [287]:
low_info = low_info_col(data)
data = data.drop(low_info, axis=1)
data.head()

Gender: 100.0% одинаковых значений


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,98,58,33,190,34.0,0.430,43,0
1,2,112,75,32,0,35.7,0.148,21,0
2,2,108,64,0,0,30.8,0.158,21,0
3,8,107,80,0,0,24.6,0.856,34,0
4,7,136,90,0,0,29.9,0.210,50,0


Задание 8.3

Попробуйте найти пропуски в данных с помощью метода isnull().

Спойлер: ничего не найдёте. А они есть! Просто они скрыты от наших глаз. В таблице пропуски в столбцах Glucose, BloodPressure, SkinThickness, Insulin и BMI обозначены нулём, поэтому традиционные методы поиска пропусков ничего вам не покажут. Давайте это исправим!

Замените все записи, равные 0, в столбцах Glucose, BloodPressure, SkinThickness, Insulin и BMI на символ пропуска. Его вы можете взять из библиотеки numpy: np.nan.

Какая доля пропусков содержится в столбце Insulin? Ответ округлите до сотых.

In [288]:
data.isnull().value_counts()
data.replace({0 : np.nan}, inplace=True)
round(data['Insulin'].isnull().value_counts(normalize=True), 2)[True]

0.49

Задание 8.4

Удалите из данных признаки, где число пропусков составляет более 30 %. Сколько признаков осталось в ваших данных (с учетом удаленных неинформативных признаков в задании 8.2)?

In [289]:
data = nul_data_col_drop(data, nul_thresh=0.3)
data

Insulin: 49.0% zero values
Outcome: 65.0% zero values
7 features with less than 30% zero values


,Pregnancies,Glucose,BloodPressure,SkinThickness,BMI,DiabetesPedigreeFunction,Age
0,6.0,98.0,58.0,33.0,34.0,0.430,43
1,2.0,112.0,75.0,32.0,35.7,0.148,21
2,2.0,108.0,64.0,NaN,30.8,0.158,21
3,8.0,107.0,80.0,NaN,24.6,0.856,34
4,7.0,136.0,90.0,NaN,29.9,0.210,50
...,...,...,...,...,...,...,...
763,5.0,139.0,64.0,35.0,28.6,0.411,26
764,1.0,96.0,122.0,NaN,22.4,0.207,27
765,10.0,101.0,86.0,37.0,45.6,1.136,38
766,NaN,141.0,NaN,NaN,42.4,0.205,29


Задание 8.5

Удалите из данных только те строки, в которых содержится более двух пропусков одновременно. Чему равно результирующее число записей в таблице?

In [290]:
new_diabetes = diabetes.copy()
new_diabetes.replace({0 : np.nan}, inplace=True)
new_diabetes = nul_data_col_drop(new_diabetes, nul_thresh=0.3)
n = new_diabetes.shape[1]
new_diabetes = new_diabetes.dropna(thresh=n-2, axis=0)
new_diabetes.shape


Insulin: 49.0% zero values
Outcome: 66.0% zero values
8 features with less than 30% zero values


(761, 8)

Задание 8.6

В оставшихся записях замените пропуски на медиану. Чему равно среднее значение в столбце SkinThickness? Ответ округлите до десятых.

In [291]:
new_diab_filled = new_diabetes.copy()

values_list = list(new_diab_filled.columns)
values_list.remove('Gender')
values = dict()

for col in values_list:
    values[col]= new_diab_filled[col].median()

new_diab_filled.fillna(values, inplace=True)
round(new_diab_filled['SkinThickness'].mean(),1)



29.1

Задание 8.7

Сколько выбросов найдёт классический метод межквартильного размаха в признаке SkinThickness?


Примечание: дальнейшая работа производится с очищенными от дубликатов, неинформативных признаков данных и пропусков.

In [292]:
outliers, cleaned = outliers_iqr(new_diab_filled, 'SkinThickness')

Число выбросов по методу Тьюки: 87
Результирующее число записей: 674


Задание 8.8

Сколько выбросов найдёт классический метод z-отклонения в признаке SkinThickness?

In [293]:
outliers, cleaned = outliers_z_score(new_diab_filled, 'SkinThickness')

Число выбросов по методу z-отклонения: 4
Результирующее число записей: 757


Задание 8.9

На приведённой гистограмме показано распределение признака DiabetesPedigreeFunction. Такой вид распределения очень похож на логнормальный, и он заставляет задуматься о логарифмировании признака. Найдите сначала число выбросов в признаке DiabetesPedigreeFunction с помощью классического метода межквартильного размаха.

Затем найдите число выбросов в этом же признаке в логарифмическом масштабе (при логарифмировании единицу прибавлять не нужно!). Какова разница между двумя этими числами (вычтите из первого второе)?

In [294]:
outliers, cleaned = outliers_iqr(data=new_diab_filled, feature='DiabetesPedigreeFunction')

Число выбросов по методу Тьюки: 29
Результирующее число записей: 732


In [295]:
outliers, cleaned = outliers_iqr(data=new_diab_filled, feature='DiabetesPedigreeFunction', log_scale=True)

Число выбросов по методу Тьюки: 0
Результирующее число записей: 761
